In [1]:
import os
os.chdir("..")
os.getcwd()

'/Users/laurasisson/gin2'

In [2]:
import json
with open("Data/single_fold.json") as f:
    annealed = json.load(f)

annealed.keys()

dict_keys(['train', 'test', 'covered_notes'])

In [3]:
import data
# Building the dataset from raw GS data is faster than loading the premade dataset from Drive.
all_train_dataset = data.make_blend_dataset(annealed["train"],annealed["covered_notes"],convert_first=True)
validate_dataset = data.make_blend_dataset(annealed["test"],annealed["covered_notes"],convert_first=True)
print(f"Train Dataset: {len(all_train_dataset):,}, Annealed Train: {len(annealed['train']):,}, Test Dataset: {len(validate_dataset):,}, Annealed Test: {len(annealed['test']):,}")

'NoneType' object has no attribute 'GetAtoms'


[13:07:53] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8


  0%|          | 0/43992 [00:00<?, ?it/s]

[13:07:56] SMILES Parse Error: syntax error while parsing: (C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC
[13:07:56] SMILES Parse Error: Failed parsing SMILES '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC' for input: '(C)C1=CN=CC(=N1)OC.CC(C)C1=CN=C(C=N1)OC.CC(C)C1=NC=CN=C1OC'


'NoneType' object has no attribute 'GetAtoms'
'NoneType' object has no attribute 'GetAtoms'


[13:07:57] SMILES Parse Error: syntax error while parsing: InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3
[13:07:57] SMILES Parse Error: Failed parsing SMILES 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3' for input: 'InChI=1/C7H8S/c1-6-4-2-3-5-7(6)8/h2-5,8H,1H3'


  0%|          | 0/39554 [00:00<?, ?it/s]

Train Dataset: 43,983, Annealed Train: 43,992, Test Dataset: 39,523, Annealed Test: 39,554


In [4]:
ex_blend = all_train_dataset[0]

In [5]:
IN_DIM = ex_blend["graph"].x.shape[1]
NOTES_DIM = ex_blend["notes"].shape[0]
IN_DIM, NOTES_DIM

(9, 77)

In [6]:
import message
import gcn

conv_configurations = [
    message.ConvConfiguration(num_convs=2, hidden_dim=32),
    message.ConvConfiguration(num_convs=3, hidden_dim=64)
]
num_hidden_layers = 2
dropout = 0.1
do_two_stage = True
num_heads = 4
num_sabs = 2
act_mode = "relu"
do_feature_norm = True
do_conv_norm = True
do_residual = True

model = gcn.GCN(
    in_features=IN_DIM,
    conv_configurations=conv_configurations,
    num_hidden_layers=num_hidden_layers,
    dropout=dropout,
    do_two_stage=do_two_stage,
    num_heads=num_heads,
    num_sabs=num_sabs,
    notes_dim=NOTES_DIM,
    act_mode=act_mode,
    do_feature_norm=do_feature_norm,
    do_conv_norm=do_conv_norm,
    do_residual=do_residual
)

model

GCN(
  (conv_blocks): ModuleList(
    (0): GINConvBlock(
      (feature_norm): BatchNorm1d(9, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (projection): Sequential(
        (0): Linear(in_features=9, out_features=32, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.1, inplace=False)
      )
      (conv): GINConv(nn=Sequential(
        (0): Linear(in_features=32, out_features=32, bias=True)
        (1): ReLU()
        (2): Linear(in_features=32, out_features=32, bias=True)
        (3): ReLU()
        (4): Linear(in_features=32, out_features=32, bias=True)
      ))
      (norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): GINConvBlock(
      (feature_norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (projection): Sequential(
        (0): Linear(in_features=32, out_features=64, bias=True)
        (1): ReLU()
        (2

In [7]:
model(ex_blend["graph"])

{'embed': tensor([[-0.1341,  0.2906,  0.6842,  0.5453, -0.0346, -0.2705,  0.7619,  0.3842,
           0.2936,  0.4820,  0.0295, -0.0357,  0.4187,  0.6615,  0.1073, -0.0824,
          -0.0150,  0.2739,  0.5397,  0.0994,  0.1316,  0.3641,  0.4759, -0.1302,
           0.2307,  0.0416,  0.2191,  0.4437,  0.3094,  0.0915,  0.2832,  0.1910,
           0.4571,  0.7700, -0.0518, -0.0541,  0.1061,  0.0078,  0.3422,  1.0358,
          -0.3024,  0.0360,  0.3692,  0.0490,  0.0472, -0.0958,  0.2567,  0.4852,
           0.0300,  0.2939, -0.1729,  0.8888,  0.8285,  0.0702,  0.6942,  0.0657,
          -0.4719,  0.0417,  0.4058, -0.2398,  0.3887, -0.0642, -0.3262,  0.5440]],
        grad_fn=<CloneBackward0>),
 'logits': tensor([[ 0.0921, -0.2621,  0.3977, -0.0582, -0.1110, -0.0489,  0.2258, -0.0937,
           0.1162, -0.2408,  0.0329,  0.2721, -0.2228,  0.0481,  0.1124, -0.0216,
          -0.2759,  0.4277,  0.1525,  0.3807, -0.0892, -0.3885, -0.1086,  0.0502,
          -0.0117, -0.2406,  0.0273, -0.03

In [9]:
model.conv_blocks[0](ex_blend["graph"].x,ex_blend["graph"].edge_index,ex_blend["graph"].x).shape

torch.Size([23, 32])